In [4]:
import pandas as pd
import os

In [8]:
dir_path = os.getcwd()
folder_path = f"{dir_path}/search_result"
file_name_list = os.listdir(folder_path)
len(file_name_list), file_name_list[0]

(11118, '17928.csv')

In [13]:
from tqdm import tqdm

all_df = pd.DataFrame()
dfs = []
for file_name in tqdm(file_name_list):
    # each file contain 10000 rows
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    dfs.append(df)
import time

start_time = time.time()
all_df = pd.concat(dfs, ignore_index=True)
print(f"--- {time.time() - start_time} seconds ---")

100%|██████████| 11118/11118 [00:32<00:00, 337.31it/s]


--- 0.7905001640319824 seconds ---


In [14]:
all_df.shape

(122776084, 3)

In [15]:
all_df.head()

,input_store_id,pred_store_id,score
0,17928,17928,0.000000
1,17928,13634,10.323901
2,17928,8232,11.813396
3,17928,16451,11.857945
4,17928,14529,12.068649


In [16]:
# save feather
all_df.to_feather("search_result.feather")